# The legacy AIX enviroment:

In [7]:
%%bash

export AIX_HOST="52.117.41.211"

export AIX_USER="ab3user"

export CMD="uname; oslevel -s; prtconf 2>/dev/null | head -5"

ssh ab3user@$AIX_HOST $CMD


AIX
7200-04-01-1939
System Model: IBM,9009-22A
Machine Serial Number: 78BABE0
Processor Type: PowerPC_POWER9
Processor Implementation Mode: POWER 9


# Checking the COBOL Compiler on AIX host:

In [63]:
%%bash

export AIX_HOST="52.117.41.211"

export AIX_USER="ab3user"

export CMD="lslpp -l cobol.rte"

ssh $AIX_USER@$AIX_HOST $CMD

  Fileset                      Level  State      Description         
  ----------------------------------------------------------------------------
Path: /usr/lib/objrepos
  cobol.rte                  5.1.0.0  COMMITTED  IBM COBOL for AIX Runtime


# Sample COBOL application (Edit the code in modern developement tools):
https://github.com/awsengineer/awsome-builder/blob/main/AWSome.cbl

In [76]:
%%bash
export AIX_HOST="52.117.41.211"
export AIX_USER="ab3user"
export APP_DIR="/home/cobol"
export COBOL_FILE="AWSome.cbl"

#cat <<EOF >$COBOL_FILE
#       IDENTIFICATION DIVISION.
#      PROGRAM-ID.  ABIII.
#
#       PROCEDURE DIVISION.
#       DisplayPrompt.
#           DISPLAY "Woohoo :) I am a COBOL code running on AIX!".
#           STOP RUN.
#EOF

git pull > /dev/null 2>&1

scp $COBOL_FILE $AIX_USER@$AIX_HOST:$APP_DIR && echo -e "\nThe COBOL code has been uploaded to the AIX host successfully."

export CMD="echo -e '\nA Quick look at the code on AIX host:\n' ;cat ${APP_DIR}/${COBOL_FILE}"

ssh $AIX_USER@$AIX_HOST $CMD


The COBOL code has been uploaded to the AIX host successfully.

A Quick look at the code on AIX host:

       IDENTIFICATION DIVISION.
       PROGRAM-ID.  ABIII.
       PROCEDURE DIVISION.
       DisplayPrompt.
           DISPLAY "Hello team! I am a COBOL code running on AIX.".
           STOP RUN.

# Compile the COBOL code on AIX:

In [77]:
%%bash
export AIX_HOST="52.117.41.211"
export AIX_USER="ab3user"
export APP_DIR="/home/cobol"
export COBOL_CODE="$APP_DIR/AWSome.cbl"
export EXEC_FILE="$APP_DIR/AWSome.exe"

ssh $AIX_USER@${AIX_HOST} "

echo -e 'Remove the exising complied app...\n'
rm -f $EXEC_FILE  2>/dev/null 

echo -e 'Compile the COBOL code...\n'

cob2 $COBOL_CODE -o $EXEC_FILE #>/dev/null

if [ -f $EXEC_FILE ]
then 
    echo -e '\nCheck the output binary file:\n'
    file $EXEC_FILE
    echo -e '\nRun the executable file:\n'
    $EXEC_FILE;  
else
    echo "Compile error" 
fi"


Remove the exising complied app...

Compile the COBOL code...

PP 5724-Z87 IBM COBOL for AIX  5.1.0 in progress ...
End of compilation 1,  program ABIII,  no statements flagged.

Check the output binary file:

/home/cobol/AWSome.exe: executable (RISC System/6000) or object module not stripped

Run the executable file:

Hello team! I am a COBOL code running on AIX.


# Run the same code on a Linux container:

In [90]:
%%bash

if [ -f AWSome.cbl ]
then 
    echo -e "\nThe contents of the Dockerfile:"
    echo -e "-------------------------------\n"
    cat Dockerfile
else
    echo "The COBOL source code (AWSome.cbl) does not exist."
fi


The contents of the Dockerfile:
-------------------------------

FROM debian:latest

RUN apt-get update && apt-get install -y open-cobol

RUN mkdir /src

COPY ./AWSome.cbl /src

RUN cd /src && cobc -free -x -o AWSome.exe AWSome.cbl

CMD "./src/AWSome.exe"


# Create the Docker image:

In [115]:
%%bash

docker build -t ezzobad/awsome-builder:ABIII .



Sending build context to Docker daemon  268.8kB
Step 1/6 : FROM debian:latest
 ---> e7d08cddf791
Step 2/6 : RUN apt-get update && apt-get install -y open-cobol
 ---> Running in a35c7277ba24
Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [121 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [51.9 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [270 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7907 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [7860 B]
Fetched 8424 kB in 2s (5203 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu cpp cpp-8 gcc gcc-8
  gnucobol libasan5 libatomic1 libbinutils libc-dev-bin libc6-dev libcc1-0
  libcob4

# Test the Docker image locally:

In [175]:
%%bash

docker image ls ezzobad/awsome-builder

echo -e "\nNow run a container based on the recently-created image:"
docker run ezzobad/awsome-builder:ABIII


REPOSITORY               TAG                 IMAGE ID            CREATED             SIZE
ezzobad/awsome-builder   ABIII               512c2bca69d5        3 hours ago         273MB

Now run a container based on the recently-created image:
Hello team! I am a COBOL code running on AIX.


# Push the image to a Docker repository:
https://hub.docker.com/repository/docker/ezzobad/awsome-builder

In [124]:
%%bash

docker push ezzobad/awsome-builder:ABIII


The push refers to repository [docker.io/ezzobad/awsome-builder]
e0ea48cb3bd2: Preparing
fbd64b537a93: Preparing
8e8bf575e045: Preparing
19d244b19bc6: Preparing
4762552ad7d8: Preparing
fbd64b537a93: Layer already exists
8e8bf575e045: Layer already exists
e0ea48cb3bd2: Layer already exists
4762552ad7d8: Layer already exists
19d244b19bc6: Layer already exists
ABIII: digest: sha256:9b9aff06eddd5ea62e562972ded40c7365b7738911f20028c0961517eb55f771 size: 1363


# Verify the Docker repository:

In [123]:
%%bash

echo -e "Delete the local image:\n"
docker image rm ezzobad/awsome-builder:ABIII --force

echo -e "\nNow run a container. This must pull the image from the repo:\n"
docker run ezzobad/awsome-builder:ABIII

Delete the local image:


Now run a container. This must pull the image from the repo:

Hello team! I am a COBOL code running on AIX.


# Openshift environment on AWS:

https://cloud.redhat.com/openshift/install/aws/installer-provisioned

AWS OpenShift Installer on Mac:
https://mirror.openshift.com/pub/openshift-v4/clients/ocp/latest/openshift-install-mac.tar.gz 
    
OpenShift Client (OC) for Mac:
https://mirror.openshift.com/pub/openshift-v4/clients/ocp/latest/openshift-client-mac.tar.gz 



# Explore the OpenShift cluster from CLI:

In [165]:
%%bash

export PATH=/home/ec2-user/anaconda3/bin:$PATH:/home/ec2-user/bin
        
oc login --server https://api.awsome-openshift-cluster.awsome-builder.com:6443 \
--username kubeadmin --password QUPhE-eRKET-GojYB-aHJKu > /dev/null 2>&1

oc new-project ab3project > /dev/null 2>&1

echo -e "\nCheck the cluster nodes:\n"
oc get nodes

echo -e "\nCheck the cluster resources:\n"
kubectl top nodes

echo -e "\nTweek the number of worker nodes (the hard way):\n"
echo 'oc edit machinesets.machine.openshift.io awsome-openshift-clus-f94d7-worker-us-west-1b -n openshift-machine-api'

echo -e "\nCheck the EC2 instances:\n"
echo 'https://tiny.amazon.com/48qywull/usweconsawsamazec2v2home'

echo -e "\nCheck the number of machines:\n"
oc get machinesets.machine.openshift.io -n openshift-machine-api

# kubectl run -it ab3 --image=ezzobad/awsome-builder:ABIII -- bash    
# kubectl exec ab3 -- ./src/AWSome.exe



Check the cluster nodes:

NAME                                         STATUS   ROLES    AGE     VERSION
ip-10-0-164-121.us-west-1.compute.internal   Ready    master   23d     v1.19.0+43983cd
ip-10-0-176-128.us-west-1.compute.internal   Ready    master   23d     v1.19.0+43983cd
ip-10-0-181-167.us-west-1.compute.internal   Ready    worker   7m21s   v1.19.0+43983cd
ip-10-0-190-130.us-west-1.compute.internal   Ready    worker   23d     v1.19.0+43983cd
ip-10-0-222-169.us-west-1.compute.internal   Ready    master   23d     v1.19.0+43983cd
ip-10-0-235-231.us-west-1.compute.internal   Ready    worker   23d     v1.19.0+43983cd

Check the cluster resources:

NAME                                         CPU(cores)   CPU%   MEMORY(bytes)   MEMORY%   
ip-10-0-164-121.us-west-1.compute.internal   602m         17%    3900Mi          27%       
ip-10-0-176-128.us-west-1.compute.internal   806m         23%    6137Mi          42%       
ip-10-0-181-167.us-west-1.compute.internal   91m          6%     

# Run a test container from the GUI Console
https://tiny.amazon.com/hjtrscbz/oautappsawsoawsologi 

# Run the COBOL app from OpenShift CLI:

In [174]:
%%bash
export PATH=/home/ec2-user/anaconda3/bin:$PATH:/home/ec2-user/bin

oc delete pod testpod --force 2>/dev/null
oc run testpod --image=ezzobad/awsome-builder:ABIII
sleep 3
echo 
oc logs testpod 

pod "testpod" force deleted
pod/testpod created

Hello team! I am a COBOL code running on AIX.


# Nest step: Automation of Docker image creation